In [167]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import random
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import os,sys
opj = os.path.join
from tqdm import tqdm
import acd
from copy import deepcopy
from model_mnist import LeNet5
from visualize import *
import dset_mnist as dset
import foolbox
sys.path.append('../trim')
from transforms_torch import transform_bandpass, tensor_t_augment, batch_fftshift2d, batch_ifftshift2d
from trim import *
from util import *
from attributions import *
from captum.attr import *
import warnings
warnings.filterwarnings("ignore")
# disentangled vae
sys.path.append('../disentangling-vae')
from collections import defaultdict
import vae_trim, vae_trim_viz
from disvae.utils.modelIO import save_model, load_model, load_metadata
from disvae.models.losses import get_loss_f

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### BTCVAE-Attr-Pen

In [193]:
args = vae_trim.parse_arguments()
args.loss = "btcvae"
args.reg_anneal = 0
args.btcvae_B = 0
args.attr_lamb = 10
name = args.loss + "_B_" + str(args.btcvae_B) + "_attr_" + str(args.attr_lamb)
args.name = name

In [204]:
args = vae_trim.parse_arguments()
args.loss = "btcvae"
args.name = name
# results dir
exp_dir = os.path.join(vae_trim.RES_DIR, args.name)

# load dataloaders
train_loader, test_loader = dset.load_data(args.batch_size, args.eval_batchsize, device)
metadata = load_metadata(exp_dir)

# load model
model = load_model(exp_dir, is_gpu=not args.no_cuda)

# L1Loss
L1Loss = torch.nn.L1Loss()

# saliency map
saliency = InputXGradient(DecoderEncoder(model))

In [212]:
inputs, labels = iter(test_loader).next()
inputs = inputs.to(device)
labels = labels.to(device)
recon_batch, latent_dist, latent_sample = model(inputs)

loss_f = get_loss_f(args.loss,
                    n_data=len(test_loader.dataset),
                    device=device,
                    **vars(args))
storer = defaultdict(list)
loss = loss_f(inputs, recon_batch, latent_dist, model.training,
                   storer, latent_sample=latent_sample)

loss = 0
s = deepcopy(latent_dist[0].detach())
# s = deepcopy(latent_sample.detach())
for i in range(model.latent_dim):
    col_idx = np.arange(model.latent_dim)!=i
    attributions = torch.div(saliency.attribute(s, target=i),s)[:,col_idx]
#     attributions = saliency.attribute(s, target=i)[:,col_idx]
    loss += L1Loss(attributions, torch.zeros_like(attributions))

In [213]:
loss.backward()

In [214]:
L1Loss(attributions, torch.zeros_like(attributions))

tensor(0.1019, device='cuda:0', grad_fn=<L1LossBackward>)

In [215]:
loss

tensor(1.1021, device='cuda:0', grad_fn=<AddBackward0>)

### BTCVAE-3

In [187]:
args = vae_trim.parse_arguments()
args.loss = "btcvae"
args.reg_anneal = 0
args.btcvae_B = 3
name = args.loss + "_B_" + str(args.btcvae_B)
args.name = name

In [188]:
args = vae_trim.parse_arguments()
args.loss = "btcvae"
args.name = name
# results dir
exp_dir = os.path.join(vae_trim.RES_DIR, args.name)

# load dataloaders
train_loader, test_loader = dset.load_data(args.batch_size, args.eval_batchsize, device)
metadata = load_metadata(exp_dir)

# load model
model = load_model(exp_dir, is_gpu=not args.no_cuda)

# L1Loss
L1Loss = torch.nn.L1Loss()

# saliency map
saliency = InputXGradient(DecoderEncoder(model))

In [189]:
inputs, labels = iter(test_loader).next()
inputs = inputs.to(device)
labels = labels.to(device)
recon_batch, latent_dist, latent_sample = model(inputs)

loss_f = get_loss_f(args.loss,
                    n_data=len(test_loader.dataset),
                    device=device,
                    **vars(args))
storer = defaultdict(list)
loss = loss_f(inputs, recon_batch, latent_dist, model.training,
                   storer, latent_sample=latent_sample)

loss = 0
s = deepcopy(latent_dist[0].detach())
for i in range(model.latent_dim):
    col_idx = np.arange(model.latent_dim)!=i
#     attributions = torch.div(saliency.attribute(s, target=i),s)[:,col_idx]
    attributions = saliency.attribute(s, target=i)[:,col_idx]
    loss += L1Loss(attributions, torch.zeros_like(attributions))

In [190]:
L1Loss(attributions, torch.zeros_like(attributions))

tensor(0.1394, device='cuda:0', grad_fn=<L1LossBackward>)

In [191]:
loss

tensor(0.8201, device='cuda:0', grad_fn=<AddBackward0>)

### BTCVAE-6

In [148]:
args = vae_trim.parse_arguments()
args.loss = "btcvae"
args.reg_anneal = 0
args.btcvae_B = 6
name = args.loss + "_B_" + str(args.btcvae_B)
args.name = name

In [149]:
args = vae_trim.parse_arguments()
args.loss = "btcvae"
args.name = name
# results dir
exp_dir = os.path.join(vae_trim.RES_DIR, args.name)

# load dataloaders
train_loader, test_loader = dset.load_data(args.batch_size, args.eval_batchsize, device)
metadata = load_metadata(exp_dir)

# load model
model = load_model(exp_dir, is_gpu=not args.no_cuda)

# L1Loss
L1Loss = torch.nn.L1Loss()

# saliency map
saliency = InputXGradient(DecoderEncoder(model))

In [150]:
inputs, labels = iter(test_loader).next()
inputs = inputs.to(device)
labels = labels.to(device)
recon_batch, latent_dist, latent_sample = model(inputs)

loss_f = get_loss_f(args.loss,
                    n_data=len(test_loader.dataset),
                    device=device,
                    **vars(args))
storer = defaultdict(list)
loss = loss_f(inputs, recon_batch, latent_dist, model.training,
                   storer, latent_sample=latent_sample)

loss = 0
s = deepcopy(latent_dist[0].detach())
for i in range(model.latent_dim):
    col_idx = np.arange(model.latent_dim)!=i
    attributions = torch.div(saliency.attribute(s, target=i),s)[:,col_idx]
#     attributions = saliency.attribute(s, target=i)[:,col_idx]
    loss += L1Loss(attributions, torch.zeros_like(attributions))

In [151]:
L1Loss(attributions, torch.zeros_like(attributions))

tensor(0.0091, device='cuda:0', grad_fn=<L1LossBackward>)

In [152]:
loss

tensor(0.6944, device='cuda:0', grad_fn=<AddBackward0>)

### BTCVAE-12

In [153]:
args = vae_trim.parse_arguments()
args.loss = "btcvae"
args.reg_anneal = 0
args.btcvae_B = 12
name = args.loss + "_B_" + str(args.btcvae_B)
args.name = name

In [154]:
args = vae_trim.parse_arguments()
args.loss = "btcvae"
args.name = name
# results dir
exp_dir = os.path.join(vae_trim.RES_DIR, args.name)

# load dataloaders
train_loader, test_loader = dset.load_data(args.batch_size, args.eval_batchsize, device)
metadata = load_metadata(exp_dir)

# load model
model = load_model(exp_dir, is_gpu=not args.no_cuda)

# L1Loss
L1Loss = torch.nn.L1Loss()

# saliency map
saliency = InputXGradient(DecoderEncoder(model))

In [155]:
inputs, labels = iter(test_loader).next()
inputs = inputs.to(device)
labels = labels.to(device)
recon_batch, latent_dist, latent_sample = model(inputs)

loss_f = get_loss_f(args.loss,
                    n_data=len(test_loader.dataset),
                    device=device,
                    **vars(args))
storer = defaultdict(list)
loss = loss_f(inputs, recon_batch, latent_dist, model.training,
                   storer, latent_sample=latent_sample)

loss = 0
s = deepcopy(latent_dist[0].detach())
for i in range(model.latent_dim):
    col_idx = np.arange(model.latent_dim)!=i
    attributions = torch.div(saliency.attribute(s, target=i),s)[:,col_idx]
#     attributions = saliency.attribute(s, target=i)[:,col_idx]
    loss += L1Loss(attributions, torch.zeros_like(attributions))

In [156]:
L1Loss(attributions, torch.zeros_like(attributions))

tensor(0.0893, device='cuda:0', grad_fn=<L1LossBackward>)

In [157]:
loss

tensor(0.3086, device='cuda:0', grad_fn=<AddBackward0>)